In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from ydata_synthetic.synthesizers import ModelParameters
from ydata_synthetic.preprocessing.timeseries.timeseries_processor import TimeSeriesDataProcessor
from ydata_synthetic.synthesizers.timeseries import TimeGAN

2022-10-01 14:21:03.603781: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-01 14:21:03.603885: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
df = pd.read_csv('../data/tum_synthetic_training_data/train.csv')
df_arr = [df[df['case']==case] for case in df['case'].unique()]


In [3]:
data = []
for df_case in df_arr:
    end_idx = 5760
    while end_idx < df_case.shape[0]:
        start_idx = end_idx - 5760
        hr = df_case['heart_rate'][start_idx:end_idx].to_numpy()
        so = df_case['symptom_onset'][start_idx:end_idx].to_numpy()
        end_idx += 288
        data.append(np.concatenate([hr[:, None], so[:, None]], axis=1))  
print(len(data))
print(data[0].shape)

4849
(5760, 2)


In [4]:
seq_len = 5760
n_seq = 2
hidden_dim=24
gamma=1

noise_dim=100
dim=64
batch_size=1

log_step = 100
learning_rate = 5e-4
retrain=True

gan_args = ModelParameters(batch_size=batch_size, lr=learning_rate, noise_dim=noise_dim, layers_dim=dim)


In [5]:
if retrain:
    synth = TimeGAN(model_parameters=gan_args, hidden_dim=hidden_dim, seq_len=seq_len, n_seq=n_seq, gamma=gamma)
    synth.train(data, train_steps=10)
    synth.save('synthesizer_hr.pkl')
else:
    synth = TimeGAN.load('synthesizer_hr.pkl')

2022-10-01 14:21:39.848987: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-10-01 14:21:39.849197: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-01 14:21:39.849287: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (MW-Thinkpad): /proc/driver/nvidia/version does not exist
2022-10-01 14:21:39.899261: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Emddeding network training:   0%|          | 0/10 [00:00<?, ?it/s]2022-10-01 14:21:56.836135: W tensorflow/core

In [ ]:
synth_data = synth.sample(len(data))